In [1]:
import numpy as np
import matplotlib.pyplot as plt


from rtmag.process.paper.analytical_field import get_analytic_b_field

from rtmag.process.paper.load import MyModel
device = "cpu"
import rtmag.process.paper.metric as m
from rtmag.process.paper.potential_field import get_potential_field
from rtmag.process.paper.opt_jit import relax

In [2]:
l = 0.3
psi = 0.25*np.pi
B = get_analytic_b_field(m=1, l=l, psi=psi)

In [3]:
np.save('analytic_b_field.npy', B)

# Potential

In [4]:
B = np.load('analytic_b_field.npy')
B.shape

(64, 64, 64, 3)

In [5]:
bottom = B[:, :, 0, :]
print(bottom.shape)

nx, ny, nz, _ = B.shape
print(nx, ny, nz)

(64, 64, 3)
64 64 64


In [6]:
bp = get_potential_field(bottom[..., 2], nz, l=0.5)

Potential Field: 100%|██████████| 128/128 [00:04<00:00, 28.27it/s]


In [ ]:
bp1 = np.load("lowlou_pot_jit.npy")
np.allclose(bp, bp1)

In [7]:
np.save("lowlou_pot_jit.npy", bp)

In [8]:
res = m.evaluate(bp, B)
print()
print(f"sigmaJ_b: {res['sigmaJ_b']:.4f}")
print(f"div_b   : {res['div_b']:.4f}")
print(f"sigmaJ_B: {res['sigmaJ_B']:.4f}")
print(f"div_B   : {res['div_B']:.4f}")

C_vec     : 0.85
C_cs      : 0.81
E_n'      : 0.42
E_m'      : 0.33
eps       : 0.62
sigmaJ_b  : 0.82
div_b     : 0.00
sigmaJ_B  : 0.02
div_B     : 0.00
sig100_b  : 81.65
div100_b  : 0.11
sig100_B  : 1.71
div100_B  : 0.02

sigmaJ_b: 0.8165
div_b   : 0.0011
sigmaJ_B: 0.0171
div_B   : 0.0002


# Optimization

In [9]:
B = np.load('analytic_b_field.npy')
bottom = B[:, :, 0, :]
print(bottom.shape)

(64, 64, 3)


In [10]:
bp = np.load("lowlou_pot_jit.npy")
bp.shape

(64, 64, 64, 3)

In [11]:
bp[:, :, 0, :] = bottom

In [12]:
from time import perf_counter

In [13]:
start_time = perf_counter()

br = relax(bp, 'lowlou_opt')

end_time = perf_counter()
total_time = end_time - start_time
print(f"Total time: {total_time:.2f}s")

nx, ny, nz =  64 64 64
Lx, Ly, Lz =  tensor(1.) tensor(1.) tensor(1.)
dx, dy, dz =  tensor(0.0159) tensor(0.0159) tensor(0.0159)
Bave =  tensor(27.1043)
maxit =  10000
mue =  tensor(2.5195e-05)
it =  0 L =  tensor(69.3502, device='cuda:0') F =  tensor(74.1130, device='cuda:0') Li =  tensor(0.1786, device='cuda:0') Fi =  tensor(1.7168, device='cuda:0')
grad L/L tensor(1., device='cuda:0')
it =  10 L =  tensor(17.1005, device='cuda:0') F =  tensor(24.6961, device='cuda:0') Li =  tensor(5.7479, device='cuda:0') Fi =  tensor(18.6640, device='cuda:0')
grad L/L tensor(3.0554, device='cuda:0')
it =  20 L =  tensor(9.3081, device='cuda:0') F =  tensor(13.1182, device='cuda:0') Li =  tensor(6.9628, device='cuda:0') Fi =  tensor(21.3233, device='cuda:0')
grad L/L tensor(0.8372, device='cuda:0')
it =  30 L =  tensor(6.4595, device='cuda:0') F =  tensor(9.8185, device='cuda:0') Li =  tensor(5.6869, device='cuda:0') Fi =  tensor(19.6391, device='cuda:0')
grad L/L tensor(0.4410, device='cuda:0')
it 

In [14]:
np.save("lowlou_opt.npy", br)

In [15]:
res = m.evaluate(np.array(br), B)
print()
print(f"sigmaJ_b: {res['sigmaJ_b']:.4f}")
print(f"div_b   : {res['div_b']:.4f}")
print(f"sigmaJ_B: {res['sigmaJ_B']:.4f}")
print(f"div_B   : {res['div_B']:.4f}")

C_vec     : 0.99
C_cs      : 0.85
E_n'      : 0.75
E_m'      : 0.45
eps       : 0.98
sigmaJ_b  : 0.12
div_b     : 0.02
sigmaJ_B  : 0.02
div_B     : 0.00
sig100_b  : 12.25
div100_b  : 1.77
sig100_B  : 1.71
div100_B  : 0.02

sigmaJ_b: 0.1225
div_b   : 0.0177
sigmaJ_B: 0.0171
div_B   : 0.0002


# AI

In [ ]:
meta_path = "D:/models/lowlou/best_model.pt"

mm = MyModel(meta_path, device=device)

In [ ]:
B = np.load('analytic_b_field.npy')
B.shape

In [ ]:
b_bottom = B[:, :, 0, :]
b_bottom = b_bottom.transpose((1, 0, 2))[None, None, ...]
print(b_bottom.shape)

In [ ]:
b = mm.get_pred_from_numpy(b_bottom)

In [ ]:
np.save("lowlou_pino.npy", b)

In [ ]:
draw(b)

In [ ]:
res = m.evaluate(b, B)
print()
print(f"sigmaJ_b: {res['sigmaJ_b']:.4f}")
print(f"div_b   : {res['div_b']:.4f}")
print(f"sigmaJ_B: {res['sigmaJ_B']:.4f}")
print(f"div_B   : {res['div_B']:.4f}")

# AI + Optimization

In [ ]:
B = np.load('analytic_b_field.npy')
b = np.load("lowlou_pino.npy")

In [ ]:
bottom = B[:, :, 0, :]
print(bottom.shape)

In [ ]:
b[:, :, 0, :] = bottom

In [ ]:
br_pino = relax(b, 'lowlou_pino_opt')

In [ ]:
np.save("lowlou_pino_opt.npy", br_pino)

In [ ]:
draw(br_pino)

In [ ]:
res = m.evaluate(br_pino, B)
print()
print(f"sigmaJ_b: {res['sigmaJ_b']:.4f}")
print(f"div_b   : {res['div_b']:.4f}")
print(f"sigmaJ_B: {res['sigmaJ_B']:.4f}")
print(f"div_B   : {res['div_B']:.4f}")